In [1]:
import pandas as pd
import numpy as np
import folium
import json
import re

## Filter Function
def mask(df, key, value):
    return df[df[key] == value]
pd.DataFrame.mask = mask

In [2]:
#####################################################
###########Input Tone, Location Raw Data#############


tone = pd.read_csv('Data/Theme/spark/theme0.csv',index_col=False,low_memory=False)
#frame = pd.read_csv('Data/Theme/theme1.csv',index_col=False,low_memory=False)

for i in range(1,2):
    path = 'Data/Theme/spark/theme' + str(i) + '.csv'
    temp = pd.read_csv(path,index_col=False,low_memory=False)
    frames = [tone,temp]
    tone = pd.concat(frames)


sourceCountry = pd.read_csv('Data/sourceCountry.csv', index_col = False, low_memory = False)
countryInfo = pd.read_csv('Data/countryInfo.csv', index_col = False, low_memory = False)
countryGeo = pd.read_csv('Data/countrygeo.csv',index_col = False,low_memory = False)
countryGeo.columns = ['fips','lat','lon','country']
countryInfo = pd.merge(countryInfo,countryGeo,on = ['country'],how = 'left')


In [11]:
tone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132639 entries, 0 to 566417
Data columns (total 5 columns):
DATE                1132639 non-null int64
SourceCommonName    1132547 non-null object
Locations           1132639 non-null object
V2Tone              1132639 non-null object
V2Themes            1017431 non-null object
dtypes: int64(1), object(4)
memory usage: 51.8+ MB


In [3]:
tone = tone.dropna()

In [4]:
######################################################
################Rename Data Columns###################

tone.columns = ['Date','Domain','Location','Tone','Theme']
tone.reset_index(inplace=True)
tone = tone[['Date','Domain','Location','Tone','Theme']]
#########Merge Country of Data Source################

tone = pd.merge(tone,sourceCountry,on = ['Domain'],how = 'left')
tone = tone.dropna()
#data.CountryName.value_counts()[:20]

In [9]:
tone.shape

(928906, 7)

In [10]:
######################################################
#################Drop Null Data#######################

tone = tone.dropna()



######################################################
###############Clean the Tone Data####################

pattern = re.compile("^\s+|\s*,\s*|\s+$")
tone['Tone'] = tone['Tone'].apply(lambda x:float(pattern.split(x)[0]))

In [11]:
######################################################
#########Country's Geo Data###########################

countryGeo.head()

,fips,lat,lon,country
0,AE,23.424076,53.847818,United Arab Emirates
1,AF,33.939110,67.709953,Afghanistan
2,AG,17.060816,-61.796428,Antigua and Barbuda
3,AI,18.220554,-63.068615,Anguilla
4,AL,41.153332,20.168331,Albania


In [12]:
############Chinese Media Data#######################

china = tone.mask('CountryName','China')
us= tone.mask('CountryName','United States')
uk = tone.mask('CountryName','United Kingdom')

In [13]:
us.shape

(225722, 7)

In [14]:
us.Tone.mean()

-1.3848755679414173

In [8]:
##########Clean the Location data mentioned in the news###############

#china['Location2'] = china.apply(lambda x: {i.split('#')[2][:2]:x['Tone'] for i in x['Location'].split(';')}, axis = 1)


In [116]:
test = us
theme = 'SCANDAL'
test = test.reset_index(inplace=False)

pattern = re.compile("^\s+|\s*,\s*|\s+$")
#tone['Tone'] = tone['Tone'].apply(lambda x:float(pattern.split(x)[0]))
#test1 = test.ix[2,'Theme'].split(';')
#pattern.split(test1[1])[0]

def themeFilter(row,theme):
    themeList = row['Theme'].split(';')
    #print themeList
    for i in range(len(themeList)):
        themeList[i] = pattern.split(themeList[i])[0]
    if theme in themeList:
        return True
    else:
        return False
    
    
#statement = test[test.apply(lambda x: themeFilter(x,theme),axis = 1)]
statement = us
statement.shape
    

(225722, 7)

In [9]:
statement = us

In [10]:
##########Clean the Location data mentioned in the news###############


def cleanTargetCountry(Loc,Tone):
    count={}
    result = {}
    for i in Loc.split(';'):
        key = i.split('#')[2][:2]
        if  key not in count.keys():
            count[key] = 1
        else:
            count[key] += 1
    totalCount = float(sum(count.values()))
    for i in Loc.split(';'):
        key = i.split('#')[2][:2]
        result[key] = count[key] / totalCount * Tone
        
    #print count
    
    return result


statement['Location1'] = statement.apply(lambda x: cleanTargetCountry(x['Location'],x['Tone']), axis = 1)
#us['Location1'] = us.apply(lambda x: cleanTargetCountry(x['Location'],x['Tone']), axis = 1)
#uk['Location1'] = uk.apply(lambda x: cleanTargetCountry(x['Location'],x['Tone']), axis = 1)

/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
############Initial China Result Dataframe##################

china_result = countryInfo[['fips_x','country']]
china_result['tone'] = 0
china_result['nb_article'] = 0
china_result = china_result.set_index('fips_x')

############Initial U.S. Result Dataframe##################
us_result = countryInfo[['fips_x','country']]
us_result['tone'] = 0
us_result['nb_article'] = 0
us_result = us_result.set_index('fips_x')

############Initial U.k. Result Dataframe##################
uk_result = countryInfo[['fips_x','country']]
uk_result['tone'] = 0
uk_result['nb_article'] = 0
uk_result = uk_result.set_index('fips_x')


#################################################################
###########Calculate Number of Articles for each target##########
#############Maintain the tone result to result df###############
'''
for index, row in statement.iterrows():
    target = row['Location1']
    for country in target.keys():
        try:
            china_result.ix[country,'tone'] += target[country]
            china_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass
'''   
for index, row in statement.iterrows():
    target = row['Location1']
    for country in target.keys():
        try:
            us_result.ix[country,'tone'] += target[country]
            us_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass
'''       
for index, row in uk.iterrows():
    target = row['Location1']
    for country in target.keys():
        try:
            uk_result.ix[country,'tone'] += target[country]
            uk_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass
        
'''

/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/i

"       \nfor index, row in uk.iterrows():\n    target = row['Location1']\n    for country in target.keys():\n        try:\n            uk_result.ix[country,'tone'] += target[country]\n            uk_result.ix[country,'nb_article'] +=1\n        except KeyError:\n            pass\n        \n"

In [12]:
#################################################################
#####################Normalizing Result##########################

## Calculate Average tone for other countries for China, US, UK

#china_result['tone'] = china_result['tone']/china_result['nb_article']
us_result['tone'] = us_result['tone']/us_result['nb_article']
#uk_result['tone'] = uk_result['tone']/uk_result['nb_article']


In [14]:
us_result.head(20)

,country,tone,nb_article
fips_x,,,
AN,Andorra,-0.020227,30
AE,United Arab Emirates,-0.146102,2316
AF,Afghanistan,-0.888717,3631
AC,Antigua and Barbuda,-0.308518,71
AV,Anguilla,0.260513,18
AL,Albania,-0.780590,347
AM,Armenia,-0.712298,389
AO,Angola,-0.408270,301
AY,Antarctica,-0.176485,212


In [129]:
us_result.sort(columns = 'tone',ascending=True)[:20]

/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,country,tone,nb_article,fips,lat,lon
148,Burundi,-1.392575,180,BI,-3.373056,29.918886
74,Bangladesh,-1.323409,749,BD,23.684994,90.356331
68,Romania,-1.218612,901,RO,45.943161,24.966760
55,Nigeria,-1.174393,1551,NG,9.081999,8.675277
151,Bosnia and Herzegovina,-1.149832,172,BA,43.915886,17.679076
119,Mali,-1.127421,316,ML,17.570692,-3.996166
135,Central African Republic,-1.104524,221,CF,6.611111,20.939444
35,Pakistan,-1.098939,2251,PK,30.375321,69.345116
88,Zimbabwe,-1.082869,519,ZW,-19.015438,29.154857
15,Turkey,-1.052600,5844,TR,38.963745,35.243322


In [123]:
##############################Filter Data######################################

## China

china_result = china_result.dropna()
#test2 = china_target[china_target['nb_article']>1000]
china_result = china_result.sort(columns = 'nb_article',ascending=False)[:200]
china_result = pd.merge(china_result,countryGeo, on = ['country'],how = 'left')
china_result = china_result.dropna()

#######Filiter china data##############
china_result = china_result[china_result['nb_article']>0]

## US
us_result = us_result.dropna()
#test2 = china_target[china_target['nb_article']>1000]
us_result = us_result.sort(columns = 'nb_article',ascending=False)[:200]
us_result = pd.merge(us_result,countryGeo, on = ['country'],how = 'left')
us_result = us_result.dropna()

#######Filiter china data##############
us_result = us_result[us_result['nb_article']>0]

## UK
uk_result = uk_result.dropna()
#test2 = china_target[china_target['nb_article']>1000]
uk_result = uk_result.sort(columns = 'nb_article',ascending=False)[:200]
uk_result = pd.merge(uk_result,countryGeo, on = ['country'],how = 'left')
uk_result = uk_result.dropna()

#######Filiter china data##############
uk_result = uk_result[uk_result['nb_article']>0]

/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:27: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [127]:
china_Map = folium.Map(location=[35.85, 104.19], zoom_start=1)
us_Map = folium.Map(location=[35.85, 104.19], zoom_start=1)
uk_Map = folium.Map(location=[35.85, 104.19], zoom_start=1)

In [128]:
import folium

'''
for index, row in test2.iterrows():
    print index
    Location = [row['lat'],row['lon']]
    name = row['country']
    R = abs(row.tone)
    if row.tone>0:
        Color = '#3186cc'
    else:
        Color = '#cc3131'
    if name == 'Turkey':
        print name
    if index>30:
        folium.CircleMarker(location=Location, radius=1000000*R,
                            popup=name,
                            color=Color,
                            fill_color=Color).add_to(Map)

'''
def draw(Map,loc,r,name):
    if r > 0.05:
        Color = '#3186cc'
    elif r < -0.05:
        Color = '#cc3131'
    else:
        Color = '#0ed148'#0ed148
    
    folium.CircleMarker(location=loc, radius=700000*abs(r),
                            popup=name+': '+str(r),
                            color=Color,
                            fill_color=Color).add_to(Map)
    
#china_result.apply(lambda x: draw(china_Map,[x['lat'],x['lon']],x['tone'],x['country']),axis = 1)
us_result.apply(lambda x: draw(us_Map,[x['lat'],x['lon']],x['tone'],x['country']),axis = 1)
#uk_result.apply(lambda x: draw(uk_Map,[x['lat'],x['lon']],x['tone'],x['country']),axis = 1)

        
    
#china_Map.save('iconTest_china.png')
#us_Map.save('iconTest_us.png')
#uk_Map.save('iconTest_uk.png')

us_Map

In [126]:
us_Map

In [65]:
uk_Map